# Reconciliating a Categorical Field 
through an OpenRefine Reconciliation Service

In [77]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import os

from heritageconnector.config import config, field_mapping
from heritageconnector.utils.sparql import get_sparql_results

import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

import requests
from collections import Counter
from tqdm import tqdm
import re
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. load data sample

In [103]:
sample_no = 100
random_state = 42

# load mimsy_people
df = pd.read_csv(config.MIMSY_PEOPLE_PATH)
for col in ['FIRSTMID_NAME', 'LASTSUFF_NAME']:
    df[col] = df[col].fillna("").astype(str)
    
df['FREETEXT'] = df['DESCRIPTION'].astype(str) + " " + df['NOTE'].astype(str)

# load people df 
people_df = df[df['GENDER'].isin(('M', 'F'))].sample(sample_no, random_state=random_state)
people_df.loc[:, 'JOINED_NAME'] = people_df['FIRSTMID_NAME'] + " " + people_df['LASTSUFF_NAME']

# load org df
org_df = df[df['GENDER'] == "N"].sample(sample_no, random_state=random_state)

print(f"loaded {len(people_df)} people records and {len(org_df)} organisation records")

loaded 100 people records and 100 organisation records


In [104]:
people_df.head(1)

,LINK_ID,PREFERRED_NAME,TITLE_NAME,FIRSTMID_NAME,LASTSUFF_NAME,SUFFIX_NAME,HONORARY_SUFFIX,GENDER,BRIEF_BIO,DESCRIPTION,NOTE,BIRTH_DATE,BIRTH_PLACE,DEATH_DATE,DEATH_PLACE,CAUSE_OF_DEATH,NATIONALITY,OCCUPATION,WEBSITE,AFFILIATION,LINGUISTIC_GROUP,TYPE,REFERENCE_NUMBER,SOURCE,CREATE_DATE,UPDATE_DATE,FREETEXT,JOINED_NAME
1055,30775,"Dunlop, Gilbert",NaN,Gilbert,Dunlop,NaN,NaN,M,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British,poster artist; illustrator,NaN,NaN,NaN,NaN,NaN,N,14-FEB-01,19-MAR-10,nan nan,Gilbert Dunlop


In [105]:
field_mapping.PEOPLE

{'LINK_ID': {'type': 'index'},
 'PREFERRED_NAME': {'PID': 'label', 'RDF': '', 'type': 'str'},
 'FIRSTMID_NAME': {'PID': 'P735', 'RDF': '', 'type': 'str'},
 'LASTSUFF_NAME': {'PID': 'P734', 'RDF': '', 'type': 'str'},
 'BIRTH_DATE': {'PID': 'P569', 'RDF': '', 'type': 'date'},
 'DEATH_DATE': {'PID': 'P570', 'RDF': '', 'type': 'date'},
 'BIRTH_PLACE': {'PID': 'P19', 'RDF': '', 'type': 'place'},
 'DEATH_PLACE': {'PID': 'P20', 'RDF': '', 'type': 'place'},
 'OCCUPATION': {'PID': 'P106', 'RDF': '', 'type': 'list (str)'},
 'DESCRIPTION': {'type': 'longstr'},
 'NOTE': {'type': 'longstr'}}

## 2. reconcile `occupation` column

In [106]:
def str_col_to_list(series, separator=";"):
    return series.fillna("").astype(str).apply(lambda i: [x.strip().lower() for x in i.split(separator)])

In [107]:
people_df['OCCUPATION_list'] = str_col_to_list(people_df['OCCUPATION'])
people_df['OCCUPATION_list']

1055                         [poster artist, illustrator]
7730                                          [medallist]
16648                                                  []
9785                         [maker of telescope replica]
6500     [typographer, type-designer, printer, publisher]
                               ...                       
11947                   [physician, surgeon, pathologist]
499                              [painter, poster artist]
13843          [optical & philosophical instrument maker]
5009                                                   []
5739                                     [conjoined twin]
Name: OCCUPATION_list, Length: 100, dtype: object

In [108]:
occupation_count = pd.Series(Counter(people_df["OCCUPATION_list"].sum())).drop(index='').sort_values(ascending=False)
print(occupation_count.head(10))

occupations_unique = occupation_count.index.tolist()
len(occupations_unique)

poster artist                    9
photographer                     8
artist                           7
inventor                         6
scientist                        5
surgeon                          4
engineer                         4
physician                        4
mathematical instrument maker    3
medallist                        3
dtype: int64


82

In [180]:
def construct_query_df(name_list, entity_type):
    """
    Creates a dataframe which can be converted into pages of queries for a Reconciliation Service API.
    """
    
    all_names = name_list.sum()
    
    # get rid of disallowed URL characters, double spaces and empty strings
    translator = str.maketrans('', '', ":?#/[]@!$&'()*+,;=")
    all_names = [i.translate(translator) for i in all_names]
    all_names = [i.replace("  ", " ") for i in all_names if i != ""]
    
    series_count = pd.Series(Counter(all_names)).sort_values(ascending=False)
    query_df = pd.DataFrame(series_count).reset_index().drop(columns=0).rename(columns={'index': 'query'})
    
    if isinstance(entity_type, str):
        query_df['type'] = entity_type
    elif isinstance(entity_type, list):
        query_df['type'] = [entity_type for _ in range(len(query_df))]
    
    return query_df

def reconcile_from_query_df(query_df, endpoint_url, page_limit=18):
    """
    Reconciles list of strings from query dataframe.
    """
    
    endpoint_url = endpoint_url + "?queries="
    
    idx_list = query_df.index.tolist()
    idx_paginated = [idx_list[i : i + page_limit] for i in range(0, len(idx_list), page_limit)]

    response_df = pd.DataFrame()
    failed_idx = []
    for page in tqdm(idx_paginated):
        try:
            json_request = query_df.loc[page, :].to_json(orient="index")
            response = requests.get(endpoint_url + json_request)
            response.raise_for_status()

            json_response = response.json()

            tempdf = pd.json_normalize(json_response).T
            tempdf = tempdf.rename(index = lambda x: x.strip('.result'), columns={0: 'response_dump'})

            response_df = response_df.append(tempdf)

        except Exception as e:
            print(f"FAILED: {page[0]}:{page[-1]}")
            print(endpoint_url + json_request)
            print(e)
    
    return response_df

In [178]:
#query_df = construct_query_df(people_df["OCCUPATION_list"], 'Q12737077')
query_df = construct_query_df(people_df["OCCUPATION_list"], ['Q28640', 'Q12737077'])
query_df.head()

,query,type
0,poster artist,"[Q28640, Q12737077]"
1,photographer,"[Q28640, Q12737077]"
2,artist,"[Q28640, Q12737077]"
3,inventor,"[Q28640, Q12737077]"
4,scientist,"[Q28640, Q12737077]"


In [208]:
def reconcile_from_query_df_post(query_df, endpoint_url, page_limit=18):
    """
    Reconciles list of strings from query dataframe.
    """
    
    #endpoint_url = endpoint_url + "?queries="
    
    idx_list = query_df.index.tolist()
    idx_paginated = [idx_list[i : i + page_limit] for i in range(0, len(idx_list), page_limit)]

    response_df = pd.DataFrame()
    failed_idx = []
    for page in tqdm(idx_paginated):
        try:
            json_request = query_df.loc[page, :].to_json(orient="index")
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
            body = "queries=" + json.dumps(json.loads(json_request))
            session = requests.Session()
            response = session.post(endpoint_url, headers=headers, data=body)
            response.raise_for_status()

            json_response = response.json()

            tempdf = pd.json_normalize(json_response).T
            tempdf = tempdf.rename(index = lambda x: x.strip('.result'), columns={0: 'response_dump'})

            response_df = response_df.append(tempdf)

        except Exception as e:
            # TODO: retry once
            print(f"FAILED: {page[0]}:{page[-1]}")
            print(body)
            print(e)
    
    return response_df

# https://wdreconcile.toolforge.org/en/api
# https://tools.wmflabs.org/openrefine-wikidata/en/api

response_df = reconcile_from_query_df_post(query_df, endpoint_url="https://wdreconcile.toolforge.org/en/api", page_limit=10)
response_df.index = response_df.index.astype(int)
response_df.head()

 44%|████▍     | 4/9 [05:49<08:01, 96.23s/it] 

FAILED: 30:39
queries={"30": {"query": "admiral", "type": ["Q28640", "Q12737077"]}, "31": {"query": "mechanical engineer", "type": ["Q28640", "Q12737077"]}, "32": {"query": "broadcaster", "type": ["Q28640", "Q12737077"]}, "33": {"query": "nautical instrument maker", "type": ["Q28640", "Q12737077"]}, "34": {"query": "instrument maker", "type": ["Q28640", "Q12737077"]}, "35": {"query": "chronophotographer", "type": ["Q28640", "Q12737077"]}, "36": {"query": "natural philosopher", "type": ["Q28640", "Q12737077"]}, "37": {"query": "neurologist", "type": ["Q28640", "Q12737077"]}, "38": {"query": "astronomer", "type": ["Q28640", "Q12737077"]}, "39": {"query": "manufacturer", "type": ["Q28640", "Q12737077"]}}
502 Server Error: Bad Gateway for url: https://wdreconcile.toolforge.org/en/api


100%|██████████| 9/9 [09:17<00:00, 61.97s/it]


response_dump
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
combined_df = pd.concat([query_df, response_df], axis=1)
combined_df

In [149]:
from fuzzywuzzy import fuzz

# TODO: add to list of matches if score == 100 or name matches

combined_df["match_name"] = ""
combined_df["match_id"] = ""

for idx, row in tqdm(combined_df.iterrows(), total=combined_df.shape[0]):
    query = row['query']
    response = row['response_dump']
    
    if not isinstance(response, float):
        names = [item['name'] for item in response]
        ids = [item['id'] for item in response]
        idx_match = [idx for idx, item in enumerate(names) if fuzz.ratio(query, item) > 90]

        if len(idx_match) >= 1:
            combined_df.loc[idx, "match_name"] = names[idx_match[0]]
            combined_df.loc[idx, "match_id"] = ids[idx_match[0]]
        if len(idx_match) > 1:
            print(query, [names[i] for i in idx_match])

100%|██████████| 81/81 [00:00<00:00, 1608.35it/s]

clergyman ['clergyman', 'clergyman']
apprentice ['apprentice', 'apprentice']
optician ['optician', 'optician']


## 3. Getting properties from field name
We can use `P1629: subject item of this property` to get the type of entity we should be searching for from our column (i.e. the one in config).

In [37]:
def get_properties_from_field(field_pid):
    query = f"""
    SELECT ?property WHERE {{
      wd:{field_pid} wdt:P1629 ?property.
    }}
    """
    
    res = get_sparql_results(config.WIKIDATA_SPARQL_ENDPOINT, query)
    
    if 'results' in res:
        bindings = res['results']['bindings']
        qids = [re.findall("(Q\d+)", item['property']['value'])[0] for item in bindings]
        
        return qids

In [38]:
get_properties_from_field(field_mapping.PEOPLE['OCCUPATION']['PID'])

['Q28640', 'Q12737077']